## Sarcasm Detector

Sometimes sarcasm is hard to detect, because it's a non-literal use of language and is context dependent. If there exists an intelligent machine learning model that knows sarcasm very well, humans can forever be saved from future misunderstandings and embarrassment.

Since sarcasm is very context dependent, collecting the "correct" data is intractable. Principle of inferability states that speakers only use sarcasm if they can be sure it will be understood by the audience. However, it's interesting to note that sarcasm is still used in social media even when the audience is so large and unfamiliar. In the virtual world, speakers instead use indicators, such as Twitter hashtags, to convey what they infer.

This is an implementation of ["Contextualized Sarcasm Detection on Twitter"](http://www.cs.cmu.edu/~nasmith/papers/bamman+smith.icwsm15.pdf).

In [1]:
import tensorflow as tf
import numpy as np
import tempfile

For features, I considered both linguistic and contextual information:
- number of intensifiers, undertones, and vowel excluded spellings (linguistic)
- number of author's followers and posts, duration on Twitter, account verification and timezone (contextual)

In [2]:
columns = [ "num_intensifiers", "num_downtoners", "num_no_vowels", "verified", "num_followers", "num_statuses", "since_created", "time_zone" ]

def is_categorical(column):
    return column == "verified" or column == "time_zone"

num_intensifiers = tf.feature_column.numeric_column(columns[0])
num_downtoners = tf.feature_column.numeric_column(columns[1])
num_no_vowels = tf.feature_column.numeric_column(columns[2])
verified = tf.feature_column.categorical_column_with_vocabulary_list(columns[3], ['True', 'False'])
num_followers = tf.feature_column.numeric_column(columns[4])
num_statuses = tf.feature_column.numeric_column(columns[5])
since_created = tf.feature_column.numeric_column(columns[6])
time_zone = tf.feature_column.categorical_column_with_hash_bucket(columns[7], hash_bucket_size=1000)

tf_columns = [ num_intensifiers, num_downtoners, num_no_vowels, verified, num_followers, num_statuses, since_created, time_zone ]

For this task, I decided to use logistic regression on streams of live tweets, using presence and absence of the hashtag "#sarcasm" to differentiate for learning. 

In [3]:
model = tf.estimator.LinearClassifier(
    feature_columns=tf_columns,
    optimizer=tf.train.FtrlOptimizer(
        learning_rate=0.1,
        l1_regularization_strength=1.0,
        l2_regularization_strength=1.0),
    model_dir=tempfile.mkdtemp())

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': '/tmp/tmptOzMhV', '_save_summary_steps': 100}


In [4]:
training_set = {}
training_labels = []
for column in columns:
    training_set.update({column: []})
    
f = open('tweets.csv', 'r')
f.readline()
for line in f:
    data = line.rstrip().split('\t')
    training_labels.append(data[-1] == 'True')
    for i in range(len(data) - 1):
        training_set[columns[i]].append(data[i] if is_categorical(columns[i]) else float(data[i]))
f.close()

In [5]:
def input_fn(d, l):
    return tf.estimator.inputs.numpy_input_fn(
        x={k: np.array(v) for k, v in d.items()},
        y=np.array(l),
        shuffle=True)

For a given data of size 100 with 10-fold cross validation, the accuracy of the model is about 0.633333334327.

In [6]:
K = 10
data_size = len(training_labels)
size = data_size / K
total_accuracy = 0
for i in range(K):
    trs = { k: v[0:(i * size)] + v[(i * size) + size:data_size] for k, v in training_set.items() }
    ts = { k: v[(i * size):((i * size) + size)] for k, v in training_set.items() }

    trsl = training_labels[0:(i * size)] + training_labels[(i * size) + size:data_size]
    tsl = training_labels[(i * size):((i * size) + size)]

    model.train(input_fn=input_fn(trs, trsl), steps=1)
    result = model.evaluate(input_fn=input_fn(ts, tsl), steps=1)

    total_accuracy += result['accuracy']

print total_accuracy / K

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmptOzMhV/model.ckpt.
INFO:tensorflow:loss = 62.3833, step = 1
INFO:tensorflow:Loss for final step: 62.3833.
INFO:tensorflow:Starting evaluation at 2018-08-23-02:03:54
INFO:tensorflow:Restoring parameters from /tmp/tmptOzMhV/model.ckpt-1
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2018-08-23-02:03:54
INFO:tensorflow:Saving dict for global step 1: accuracy = 0.888889, accuracy_baseline = 1.0, auc = 1.0, auc_precision_recall = 0.0, average_loss = 161.27, global_step = 1, label/mean = 0.0, loss = 1451.43, prediction/mean = 0.111111
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmptOzMhV/model.ckpt-1
INFO:tensorflow:Saving checkpoints for 2 into /tmp/tmptOzMhV/model.ckpt.
INFO:tensorflow:loss = 3.14673e+08, step = 2
INFO:tensorflow:Loss for final step: 3.14673e+08.
INFO:tensorflow:Starting evaluation at 2018-08-23-02:03:57
I

INFO:tensorflow:Saving dict for global step 10: accuracy = 0.0, accuracy_baseline = 1.0, auc = 0.0, auc_precision_recall = 1.0, average_loss = 6.08789e+06, global_step = 10, label/mean = 1.0, loss = 5.4791e+07, prediction/mean = 0.0
0.633333334327
